# Mask R-CNN
The original Mask R-CNN implenetation is too old to run on one system with modern CUDA drivers etc. (even some forks whcih purport to have updated it for Tensorflow 2.x are not working for me), so I opted to use the `torchvision` version. The goal here is to make it as easy as possible for someone to reproduce this work, and having to deal with the nightmare that is out-of-date tensorflow and CUDA is not tenable. 

This script adapts a couple key functions (like the data loader and remove_overlaps) from the original train_maskrcnn script from Cellpose. 

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import time, os, sys
from tifffile import imread

### Define data loader

In [ ]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image
from cellpose import io, transforms
from omnipose.utils import format_labels
import skimage.io
from tifffile import imread
import omnipose

from pathlib import Path
def getname(path,suffix='_masks'):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')

class BacteriaDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        mask_filter = '_masks'
        img_filter = ''
        img_names = io.get_image_files(root,mask_filter,img_filter=img_filter,look_one_level_down=True)
        mask_names = io.get_label_files(img_names, mask_filter, img_filter=img_filter)
#         self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
#         self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        self.imgs = sorted(img_names,key=getname)
        self.masks = sorted(mask_names,key=getname)

    def __getitem__(self, idx):
#         print(idx)
        # load images and masks
        img_path = os.path.join(self.root, self.imgs[idx])
        mask_path = os.path.join(self.root, self.masks[idx])
        img = skimage.io.imread(img_path)#.convert('RGB')
        img = np.stack([omnipose.utils.normalize99(img)*(2**8-1)]*3,axis=-1).astype(np.uint8)
        img = Image.fromarray(img) # must convert to uint8 for pil  
        mask = imread(mask_path)
        mask = (np.array(mask))
        
        # instances are encoded as different numbers
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the integer-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            if xmax!=xmin and ymax!=ymin:
                boxes.append([xmin, ymin, xmax, ymax])
            else:
                print('uh oh',idx,obj_ids[i])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.bool)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        

        if self.transforms is not None:
            img, target = self.transforms(img, target)
#         torch.cuda.empty_cache()
        return img, target

    def __len__(self):
        return len(self.imgs)

### Define model

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=False,pretrained_backbone=True)
    
    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features

    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)
    model.roi_heads.detections_per_img = 1000
    model.roi_heads.nms_thresh = 0.7
    return model

### Define training augmentations
Notably, only random flipping is implemented here, as I beleive was the case for the original Mask R-CNN tensorflow implementation. 

In [ ]:
# The vision/references/detection/ folder needs to be available in the same directory as this notebook

from engine import train_one_epoch, evaluate
import utils
import transforms as T

def get_transform(train):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    transforms.append(T.ToTensor())
    if train:
        # during training, randomly flip the training images
        # and ground-truth for data augmentation
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

### Initialize training and test datasets

In [ ]:
#error without lowering this
num_workers = 0

# use our dataset and defined transformations
traindir = '/home/kcutler/DataDrive/omnipose_all/phase/train_sorted'
testdir = '/home/kcutler/DataDrive/omnipose_all/phase/test_sorted'
# traindir = '/home/kcutler/DataDrive/omnipose_maskrcnn/train'
# testdir = '/home/kcutler/DataDrive/omnipose_maskrcnn/test'
dataset = BacteriaDataset(traindir, get_transform(train=True))
dataset_test = BacteriaDataset(testdir, get_transform(train=False))
# split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=2, shuffle=True, num_workers=num_workers,
    collate_fn=utils.collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=num_workers,
    collate_fn=utils.collate_fn)

In [ ]:
# check to be sure it is correct
im,t = dataset[52]
im = dataset.__getitem__(2)[0]
plt.imshow(im[0])
plt.axis('off')

### Define model and training parameters

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# our dataset has two classes only - background and person
num_classes = 2

# get the model using our helper function
model = get_instance_segmentation_model(num_classes)
# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

### Train
Using recommended/default parameters. 

In [ ]:
clean = 0
if clean:
    num_epochs = 200
    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=20)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
    #     evaluate(model, data_loader_test, device=device)

### Save model

In [ ]:
modeldir = '/home/kcutler/DataDrive/maskrcnn/bacterialtrain200epochs_1000detections_per_img_v2'
if clean:
    torch.save(model, modeldir)

### Evaluate model
Test it our on a single image first

In [ ]:
# device =  torch.device('cpu')
model = torch.load(modeldir).to(device)

# pick one image from the test set
img, _ = dataset_test[0]
# put the model in evaluation mode
model.eval()
# model.roi_heads.detections_per_img = 1000 #not needed 
model.roi_heads.nms_thresh = 1
# model.roi_heads.score_thresh
with torch.no_grad():
    prediction = model([img.to(device)])

### Reconstruct cell masks
Mask R-CNN predicts bounding boxes and cell probability within those bounding boxes, along with a confidence score form 0-1. The output is sorted from high to low scores. There are two way to use this information to reconstruct labels:
1. Loop over all masks and append to a label matrix with incrementing labels. I optimized this a little bit by appending the highest scores last, thereby overwriting low-confidence labels with the higher confidence ones. 
2. Same as above, but instead of just appending, send any overlapping pixels to the closest mask. The difference here is that proximity determines the overwriting of pixels rather than confidence. 

You could imagine some more sophisticated permutations on these themes, but as you will see, the output is too poor to be meaningfully rescued. 

In [ ]:
scores = prediction[0]['scores'].cpu().numpy() # outputs in descending order
cutoff = np.percentile(scores,25)

In [ ]:
# Method (1)

im = img.permute(1, 2, 0).byte().numpy()
print(im.shape)
r = np.zeros(im.shape[:2])
l=0

print(scores.shape)
inds = np.where(scores>cutoff)[0]
for j in np.flip(inds):
    l += 1
    m = (prediction[0]['masks'][j, 0].cpu())>0.9
    r[m] = l

In [ ]:
# method (2)
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

def remove_overlaps(masks, cellpix, medians):
    """ replace overlapping mask pixels with mask id of closest mask
        masks = Nmasks x Ly x Lx
    """
    masks=masks.copy()
    overlaps = np.array(np.nonzero(cellpix>1.5)).T
    dists = ((overlaps[:,:,np.newaxis] - medians.T)**2).sum(axis=1) #square distances 
    tocell = np.argmin(dists, axis=1) # index of closest mask
    masks[:, overlaps[:,0], overlaps[:,1]] = 0 #zero out all overlapping pixels
    masks[tocell, overlaps[:,0], overlaps[:,1]] = 1 # put those pixels in

    # labels should be 1 to masks.shape[0]
    masks = masks.astype(int) * np.arange(1,masks.shape[0]+1,1,int)[:,np.newaxis,np.newaxis]
    masks = masks.sum(axis=0)
    
    # # print out variable size
    # for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
    #                          key= lambda x: -x[1])[:10]:
    #     print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
    
    return masks

In [ ]:
# the masks is actually a list of nmasks.Ly.Lx
# 0.9 is a good cutoff just based on the histogram. Could use something more sophisticated like hysteresis thresholding. 
overlapping_masks_test = np.stack([np.array((prediction[0]['masks'][j, 0].cpu())>0.9) for j in inds],axis=0)
print(len(overlapping_masks_test),overlapping_masks_test.shape)

medians_test = []
for mask in overlapping_masks_test:
   
    ypix, xpix = np.nonzero(mask)
    medians_test.append((np.array([ypix.mean(), xpix.mean()])))

labels_test = np.int32(remove_overlaps(overlapping_masks_test, overlapping_masks_test.sum(axis=0), np.array(medians_test)))


In [ ]:
cellpix_test = np.stack([np.array((prediction[0]['masks'][j, 0].cpu())>0.9) for j in inds],axis=0).sum(axis=0)
cellpix = overlapping_masks.sum(axis=0)
np.unique(cellpix_test),np.unique(cellpix)

In [ ]:
# plt.imshow(ncolor.label(cellpix,max_depth=20))
plt.imshow(cellpix>1.5)

In [ ]:
# overlapping_masks.sum(axis=0).shape
import ncolor
plt.imshow(np.hstack((ncolor.label(labels_test,max_depth=20),ncolor.label(np.uint32(r)))),interpolation='none')
plt.axis('off')
# plt.imshow(labels,interpolation='none')
print(len(np.unique(labels_test)),len(np.unique(r)))

### Batch process

Looks like method (2) is slightly better, so I'll use that for the entire dataset. Ran out of VRAM for larger images so I had to resort to running on CPU. Turns out it was faster than running on GPU? Not sure how... but regardless, both still very slow compared to more modern methods. 

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device =  torch.device('cpu')
model = torch.load(modeldir).to(device)

In [ ]:
import fastremap 

savedir = '/home/kcutler/DataDrive/omnipose_all/bact_phase_comparison/maskrcnn'
io.check_dir(savedir)

nimg = len(dataset_test)
final_masks = [[] for i in range(nimg)]


In [ ]:

# for i in range(nimg):
# for i in range(55,nimg):
for i in [0]:
    entry, path = dataset_test[i], dataset_test.masks[i]
    print(getname(path))
    img = entry[0]
    # put the model in evaluation mode
    model.eval()
    model.roi_heads.nms_thresh = 1
    # model.roi_heads.score_thresh
    with torch.no_grad():
        prediction = model([img.to(device)])
        
    scores = prediction[0]['scores'].detach().cpu().numpy() # outputs in descending order

    cutoff = np.percentile(scores,25)
    inds = np.where(scores>cutoff)[0]
    
  
    overlapping_masks = [] #[np.zeros(img.shape[-2:],dtype=np.uint8)]
    medians = [] #[[np.nan]*2]
    drop = []
    for j in inds:
        pred = np.array((prediction[0]['masks'][j, 0].detach().cpu()))
        bin0 = pred>0.9 # somewhat arbitrary cutoff
        if np.any(bin0):
            overlapping_masks.append(bin0)
            pix = np.nonzero(bin0)
            medians.append((np.array([p.mean() for p in pix])))
        else:
            drop.append(j)


    if len(overlapping_masks)>0:
        overlapping_masks = np.stack(overlapping_masks,axis=0)
        del prediction # free up memory, remove_overlaps uses a ridiculous amount for large images, if >system then kernel restarts 
        labels = np.int32(remove_overlaps(overlapping_masks, overlapping_masks.sum(axis=0), np.array(medians)))
        labels = fastremap.refit(labels)
    else:
        labels = np.zeros(img.shape[-2:],dtype=np.uint8)
        print('No masks found.')



#     overlapping_masks = np.stack([np.array((prediction[0]['masks'][j, 0].cpu())>0.9) for j in inds],axis=0)
#     medians = []
#     for mask in overlapping_masks: #runtime warning harmless, but see below
#         pix = np.nonzero(mask)
#         medians.append((np.array([p.mean() for p in pix])))

#     del prediction # free up memory, remove_overlaps uses a ridiculous amount for large images, if >system then kernel restarts 
#     labels = np.int32(remove_overlaps(overlapping_masks, overlapping_masks.sum(axis=0), np.array(medians)))
#     labels = fastremap.refit(labels)
    
    final_masks[i] = labels
    io.imsave(os.path.join(savedir,getname(path)+'_masks.tif'),labels)
    print(i,'{}% done'.format((i+1)/nimg*100))


In [ ]:
# plt.imshow(np.hstack((ncolor.label(labels,max_depth=20),ncolor.label(np.uint32(r)))),interpolation='none')
# plt.imshow(labels)
pred = np.array((prediction[0]['masks'][0, 0].detach().cpu()))
bin0 = pred>0.9 # somew
# # labels?

plt.imshow(bin0)
np.any(bin0)
# medians

In [ ]:
drop

In [ ]:
# plt.imshow(overlapping_masks[0])
plt.imshow(labels)

In [ ]:
    # This alternative branch attemps to get rid of all empty masks, but it breaks the mask recontruction, 
    # not sure yet why 
    
#     overlapping_masks = []
#     for j in inds:
#         pred = np.array((prediction[0]['masks'][j, 0].detach().cpu()))
#         bin0 = pred>0.9 # somewhat arbitrary cutoff
#         if np.any(bin0):
#             overlapping_masks.append(bin0)
    
#     if len(overlapping_masks)>0:
#         medians = []
#         for k,mask in enumerate(overlapping_masks):

#             pix = np.nonzero(mask)
#             medians.append((np.array([p.mean() for p in pix])))
        
#         overlapping_masks = np.stack(overlapping_masks,axis=0)

#         del prediction # free up memory, remove_overlaps uses a ridiculous amount for large images, if >system then kernel restarts 
#         labels = np.int32(remove_overlaps(overlapping_masks, overlapping_masks.sum(axis=0), np.array(medians)))
#         labels = fastremap.refit(labels)
        
#     else:
#         labels = np.zeros(img.shape[-2:],dtype=np.uint8)
#         print('No masks found.')
medians_test

In [ ]:
np.all(np.array((prediction[0]['masks'][j, 0].cpu()))==np.array((prediction[0]['masks'][j, 0].detach().cpu())))

In [ ]:
[np.all(overlapping_masks_test[k]==overlapping_masks[k]) for k in range(20)]

In [ ]:
for k in range(20):
# plt.imshow(overlapping_masks_test[k]==overlapping_masks[k],interpolation='None')
    print(np.any(overlapping_masks_test[k]),np.any(overlapping_masks[k]))

In [ ]:
medians[0]==medians_test[0]

In [ ]:
np.all(overlapping_masks.sum(axis=0)==overlapping_masks_test.sum(axis=0))

In [ ]:
plt.

In [ ]:
overlapping_masks.shape,overlapping_masks_test.shape

In [ ]:
np.array(medians).shape, np.array(medians_test).shape

In [ ]:
def remove_overlaps(masks, cellpix, medians):
    """ replace overlapping mask pixels with mask id of closest mask
        masks = Nmasks x Ly x Lx
    """
    masks=masks.copy()
    overlaps = np.array(np.nonzero(cellpix>1.5)).T
    dists = ((overlaps[:,:,np.newaxis] - medians.T)**2).sum(axis=1) #square distances 
    print(overlaps.shape)
    tocell = np.argmin(dists, axis=1) # index of closest mask, BUG with nans, gave 172, any dropped indices work
    # print(dists.shape,tocell.shape,dists[0:2,0:1],tocell)
    masks[:, overlaps[:,0], overlaps[:,1]] = 0 #zero out all overlapping pixels
    masks[tocell, overlaps[:,0], overlaps[:,1]] = 1 # put those pixels in

    # labels should be 1 to masks.shape[0]
    masks = masks.astype(int) * np.arange(1,masks.shape[0]+1,1,int)[:,np.newaxis,np.newaxis]
    print(masks.shape)
    masks = masks.sum(axis=0)
    
#     # print out variable size
#     for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
#                              key= lambda x: -x[1])[:10]:
#         print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

    return masks, dists, tocell

m1,d1,t1 = remove_overlaps(overlapping_masks, overlapping_masks.sum(axis=0), np.array(medians))
m2,d2,t2= remove_overlaps(overlapping_masks_test, overlapping_masks_test.sum(axis=0), np.array(medians_test))


In [ ]:
# [np.all(d1[:,k]==d2[:,k]) for k in range(20)] # dists the same because medians and cellpix the same
# np.any(overlapping_masks_test[-1])
len(t1),len(t2),fastremap.unique(t1),fastremap.unique(t2)

In [ ]:
np.arange(1,overlapping_masks.shape[0]+1,1,int)[:,np.newaxis,np.newaxis].shape

In [ ]:
# plt.imshow(m1)
np.nanargmin(d2, axis=1)

In [ ]:
medians_test[172]

In [ ]:
# d2[:,172]
t2

In [ ]:
plt.imshow(m2)

In [ ]:
drop